In [71]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re

In [135]:

driver = webdriver.Chrome()             
url = 'https://motoshare.eu/es-es'
driver.get(url)
driver.maximize_window()  
driver.implicitly_wait(20)
driver.find_element('css selector','body > div:nth-child(23) > div > div > button > svg').click()
driver.implicitly_wait(20)
driver.find_element('xpath','//*[@id="ES-AN"]').click()


In [136]:
pagina = 1
ciudad = 'Andalucia'
url = 'https://motoshare.eu/es-es/rent-motorcycle/province/andalucia'


In [148]:
def carga_datos_motos(url):
    """Cargar datos de vehículos de una página web de alquiler de motocicletas.

    Realiza una solicitud HTTP a la URL especificada y, si la solicitud es exitosa,
    analiza el contenido HTML para extraer información sobre modelos de vehículos,
    precios y lugares disponibles. La función devuelve esta información en un 
    DataFrame de pandas.

    Args:
        url (str): La URL de la página web de alquiler de motocicletas.

    Returns:
        pd.DataFrame: Un DataFrame que contiene tres columnas: 'Modelo', 
        'Precio' y 'Lugares'. Si la solicitud falla o no se encuentran 
        vehículos, se devuelve un DataFrame vacío.
    """
    res = requests.get(url,verify=False)
    
    if res.status_code == 200:
        print("Solicitud exitosa")
    else:
        print("Error en la solicitud")
        return pd.DataFrame()  

    sopa = BeautifulSoup(res.content, "html.parser")
  
    lista_vehiculos = sopa.find_all("div", {'class':'MotorcyclePlate_block_2db879'})  # Ajusta según la clase correcta
    print(lista_vehiculos)
 
    modelos = []
    precios = []
    Lugares = []

    for vehiculo in lista_vehiculos:
        modelo_elemento = vehiculo.find('div',{'class':'MotorcyclePlate_name_679f32'})  
        precio_elemento = vehiculo.find('div', {'class':'MotorcyclePlate_price-container_af1c84'})  
        lugar_elemento = vehiculo.find('div', {'class':'MotorcyclePlate_city_dc8966'})            

        modelo = modelo_elemento.get_text(strip=True) if modelo_elemento else None
        precio = precio_elemento.get_text(strip=True) if precio_elemento else None
        lugar = lugar_elemento.get_text(strip=True) if precio_elemento else None
        
        modelos.append(modelo)
        precios.append(precio)
        Lugares.append(lugar)
    
    
    df = pd.DataFrame({
        'Modelo': modelos,
        'Precio': precios,
        'Lugares':Lugares
    })

    return df


df_vehiculos = carga_datos_motos(url)


df_vehiculos.head(2)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'motoshare.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Solicitud exitosa
[<div class="MotorcyclePlate_block_2db879"><a class="Link_root_dd2cae desktop Link_size_big_fe8d0c Link_weight_bold_1a119b view_normal Link_pseudo_573f48 Link_color_dark_42caa9" href="/es-es/rent-motorcycle/bmw/2685"><span class="Link_text_6aafc8"><div class="MotorcyclePlate_image_ddfce2"></div><div class="MotorcyclePlate_name-and-price_0e365c"><div class="MotorcyclePlate_name_679f32">BMW R 1200 GS</div><div class="MotorcyclePlate_price-container_af1c84"><div class="MotorcyclePlate_price_af9a84"><div class="MotorcyclePlate_euro_708a3a">€</div>130</div><div class="MotorcyclePlate_period_e1920b">Por día</div></div></div><div class="MotorcyclePlate_footer_49f68a"><div class="Grid_row_f6c973 Grid_row_desktop_95c048 Grid_row_vAlign_start_98237a Grid_row_hAlign_start_14cc35"><div class="Grid_column_5962cc"><div class="MotorcyclePlate_city_dc8966">Granada</div></div><div class="Grid_column_5962cc"><div class="MotorcyclePlate_rating_bb3c31"><div class="MotorcyclePlate_rating-

,Modelo,Precio,Lugares
0,BMW R 1200 GS,€130Por día,Granada
1,Suzuki Burgman,€39Por día,Málaga


In [149]:
df_vehiculos.to_csv('Datos/vehiculos_malaga.csv')